# Transformer的PyTorch从头实现

## 0. 依赖和常量

### 0.1 依赖

此项目使用了PyTorch，请参考以下页面安装PyTorch：
[https://pytorch.org/get-started/locally/](https://pytorch.org/get-started/locally/)

除了PyTorch外，此项目还需要以下Python库：
* torchtext

可以使用以下pip命令安装它们：
```
pip install torchtext
```

此项目使用了spaCy tokenizer，需要使用以下命令安装spaCy语言包de_core_news_sm和en_core_web_sm：
```
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm
```

In [1]:
# Python imports.
from enum import Enum
import os

# Third-party imports.
import torch
import torchtext

### 0.2 常量

In [2]:
# Define dataset types.
class Dataset(Enum):
    MULTI30K = 1

# Define which dataset to use.
DATASET = Dataset.MULTI30K

## 1. 数据准备

我们使用[IWSLT 2017](https://wit3.fbk.eu/2017-01)中的英语翻德语数据集。torchtext.datasets集成了这个数据集，参考[文档](https://pytorch.org/text/stable/datasets.html#iwslt2017)。

其中，train、dev和test数据集各自的规模如下：
* train：206112
* dev: 888
* test: 1568

### 1.1 辅助函数定义

In [3]:
# 计算数据集中数据的数量。
def count(dataset):
    count = 0
    for item in dataset:
        count += 1
    return count

# 显示数据集的前五条数据。
def display_samples(dataset, num):
    iterator = iter(dataset)
    try:
        for i in range(num):
            print(f'{i+1}: {next(iterator)}')
    except StopIteration:
        print(f'[Error] Size of dataset is smaller than {num}.')

### 1.2 数据加载

加载train、dev和test数据集。这些数据集的数据类型为torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe。

In [4]:
# 加载train、dev和test数据集。
train, dev, test = torchtext.datasets.IWSLT2017(language_pair=('en', 'de'))

# 打印train、dev和test数据集。
print(f'Number of items in train dataset: {count(train)}')
print(f'Number of items in dev dataset: {count(dev)}')
print(f'Number of items in test dataset: {count(test)}')

# 打印train数据集前五条数据。
print('Samples from train dataset:')
display_samples(train, 5)

Number of items in train dataset: 206112
Number of items in dev dataset: 888
Number of items in test dataset: 1568
Samples from train dataset:
1: ('Thank you so much, Chris.\n', 'Vielen Dank, Chris.\n')
2: ("And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.\n", 'Es ist mir wirklich eine Ehre, zweimal auf dieser Bühne stehen zu dürfen. Tausend Dank dafür.\n')
3: ('I have been blown away by this conference, and I want to thank all of you for the many nice comments about what I had to say the other night.\n', 'Ich bin wirklich begeistert von dieser Konferenz, und ich danke Ihnen allen für die vielen netten Kommentare zu meiner Rede vorgestern Abend.\n')
4: ('And I say that sincerely, partly because  I need that.\n', 'Das meine ich ernst, teilweise deshalb -- weil ich es wirklich brauchen kann!\n')
5: ('Put yourselves in my position.\n', 'Versetzen Sie sich mal in meine Lage!\n')


### 1.3 数据tokenization

我们使用[spaCy](https://spacy.io/)库来进行tokenizer。spaCy库有多种语言的tokenization，每种语言包含多个pipeline。一种语言的不同pipeline所适用的场景不同，比如中文的pipeline zh_core_web_sm是使用web数据训练的小型通用pipeline，而zh_core_web_lg则是使用web数据训练的大型通用pipeline。可以在这篇文档中找到spaCy pipeline的命名规则、所支持的语言以及每种语言包含的pipeline。

torchtext提供了一个方便的创建spaCy tokenizer的函数：torchtext.data.get_tokenizer。它支持包含spaCy在内的多种tokenization库：basic_english、revtok、subword、spacy、moses。

在以下代码中，我们使用pipeline en_core_web_sm来对英语句子进行tokenization，使用pipeline de_core_news_sm来对德语句子进行tokenization。

In [5]:
# 加载spaCy中英语和德语的tokenizer。
tokenizer_en = torchtext.data.get_tokenizer('spacy', language='en_core_web_sm')
tokenizer_de = torchtext.data.get_tokenizer('spacy', language='de_core_news_sm')

# 打印train数据集中第一条数据的tokenization结果。
iterator_train = iter(train)
sentence = next(iterator_train)
sentence_en = sentence[0]
sentence_de = sentence[1]
print(f'{sentence_en} -> {tokenizer_en(sentence_en)}')
print(f'{sentence_de} -> {tokenizer_de(sentence_de)}')

Thank you so much, Chris.
 -> ['Thank', 'you', 'so', 'much', ',', 'Chris', '.', '\n']
Vielen Dank, Chris.
 -> ['Vielen', 'Dank', ',', 'Chris', '.', '\n']


### 1.4 创建vocabulary

在创建vacabulary时，我们考虑四种特殊token：
* `<s>`: 句首token，表示一个句子的开始。
* `</s>`：句尾token，表示一个句子的结束。
* `<blank>`：空白token，用于padding。
* `<unk>`：未知token，表示不在vocabulary中。

In [6]:
def yield_tokens(dataset, tokenizer, index):
    for item in dataset:
        yield tokenizer(item[index])

def create_vocabs(dataset, tokenizer_source, tokenizer_target):
    vocab_source = torchtext.vocab.build_vocab_from_iterator(
        yield_tokens(dataset, tokenizer_source, 0),
        specials=['<s>', '</s>', '<blank>', '<unk>'],
        min_freq=2,
    )
    vocab_target = torchtext.vocab.build_vocab_from_iterator(
        yield_tokens(dataset, tokenizer_target, 1),
        specials=['<s>', '</s>', '<blank>', '<unk>'],
        min_freq=2,
    )

    vocab_source.set_default_index(vocab_source['<unk>'])
    vocab_target.set_default_index(vocab_source['<unk>'])

    return vocab_source, vocab_target

# 优先从temp/vocabs.pt中读取vocabulary，如果文件不存在，那么重新计算并存储到文件中。
# 从文件读取vocabulary比重新计算快得多，在我的计算机上，重新计算花费24秒，重新读取花费0.1秒。
def load_or_create_vocabs():
    if not os.path.exists('temp/vocabs.pt'):
        vocab_source, vocab_target = create_vocabs(train+dev+test, tokenizer_en, tokenizer_de)
        if not os.path.exists('temp'):
            os.makedirs('temp')
        torch.save((vocab_source, vocab_target), 'temp/vocabs.pt')
    else:
        vocab_source, vocab_target = torch.load('temp/vocabs.pt')
    return vocab_source, vocab_target

vocab_source, vocab_target = load_or_create_vocabs()

# 打印输入数据集与输出数据集的vocabulary规模。
print(f'Number of vocabularies in the source language: {len(vocab_source)}')
print(f'Number of vocabularies in the target language: {len(vocab_target)}')

Number of vocabularies in the source language: 37726
Number of vocabularies in the target language: 62261


### 1.5 句子转换为index

In [7]:
def collate_batch(
    batch,
    tokenizer_source,
    tokenizer_target,
    vocab_source,
    vocab_target,
    padding_max = 128,
    padding_value = 2,
):
    # 在上面的实现中，source和target的<s>与</s>所对应的index应该是一致的，但为了逻辑的自洽性，我们在这里不做此假设。
    bos_source = torch.tensor([vocab_source['<s>']]) # Beginning of source sentence.
    eos_source = torch.tensor([vocab_source['</s>']]) # End of source sentence.
    bos_target = torch.tensor([vocab_target['<s>']]) # Beginning of target sentence.
    eos_target = torch.tensor([vocab_target['</s>']]) # End of target sentence.

    # source_list中的每一项为经过padding操作的原句子的index tensor。
    source_list = []
    # target_list中的每一项为经过padding操作的目标句子的index tensor。
    target_list = []

    for sentence_source, sentence_target in batch:
        # 将句子转换为index tensor。
        indices_source = torch.cat([bos_source, vocab_source(tokenizer_source(sentence_source)), eos_source])
        indices_target = torch.cat([bos_target, vocab_target(tokenizer_target(sentence_target)), eos_target])

        # 将句子的index tensor进行padding操作，从而使所有句子的长度一致。
        source_list.append(
            torch.nn.functional.pad(
                indices_source,
                pad=(0, padding_max-len(indices_source)),
                value=padding_value,
            )
        )
        target_list.append(
            torch.nn.functional.pad(
                indices_target,
                pad=(0, padding_max-len(indices_target)),
                value=padding_value,
            )
        )
    
    # 将source_list转换为一个大的tensor，其维度为(len(batch), padding_max)。
    source = torch.stack(source_list)
    # 将target_list转换为一个大的tensor，其维度为(len(batch), padding_max)。
    target = torch.stack(target_list)

    return source, target